# Natan Weitzman HW8 - MAE335 - Milestone 2

In Milestone 2 we aim to choose pipe and path that would minimize the 10 year cost of intalling and running the pump system (necessary minimum flow rate is 23 L/day per plant where we have 3000 plants). We assume:
1. Steady state
2. Uniform properties
3. ρ=constant
4. Gauge Pressure
5. Neglect minor losses in straight section
6. Gauge pressure at entrance and exit
7. Velocity of Lake Erie decreasing in height is approximately 0
8. Estimate minor loss coefficient due to entrance net to be K=1.765 (assuming porosity of 54.7%)

Source for 8: https://www.researchgate.net/publication/303602004_Mesh_Screen_Application_for_Noise_Reduction_of_Landing_Gear_Strut

Also assume we used regular 90 degree and regular 45 degree elbows with minor loss coefficient K of 0.9 and 0.4 respectively according to the source below.

Source: https://www.caee.utexas.edu/prof/kinnas/319LAB/notes13/Table10.5.PDF

We are given that pressure of the pipe exiting and entering the pipe is the same, and according to assumption 7, and assumption 6, thus (where η symbolizes the overall efficiency of the pump and motor):

\begin{align}
\left( \frac{P}{\rho g} + \frac{V^2}{2g} + z \right)_{\text{in}} + h_p\eta &= \left( \frac{P}{\rho g} + \frac{V^2}{2g} + z \right)_{\text{out}} + h_{\text{losses}}
\end{align}

\begin{equation*}
\downarrow
\end{equation*}


\begin{align}
h_p &= \frac{\left( \frac{V^2}{2g} + z_{\text{out}} - z_{\text{in}} + h_{\text{losses}} \right)}{\eta}
\end{align}



Total head losses is given as:

\begin{align}
h_{\text{losses}} &= h_{\text{major}} + h_{\text{minor}} \\
&= \frac{V^2}{2g} \left( f \frac{L}{d} + K \right)
\end{align}


In terms of flow rate it is:
\begin{align}
h_{\text{losses}} &= \frac{Q^2}{2gA^2} \left( f \frac{L}{d} + K \right)
\end{align}

Also we will define Reynold's number:
\begin{equation}
Re = \frac{\rho V d}{\mu}
\end{equation}

Below is the map with different paths we chose according to various paths drawn in figure:

![My Image](image.jpeg)

Let us define all the constant parameters and import libraries for the calculation:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

# Constants
efficiency = 0.95 * 0.87  # motor times pump efficiency -> overall efficiency
g = 9.81  # gravitational acceleration, m/s^2
z_in= -1.5 # in m
z_out= 20 # in m

# every 3.15 units on map is equivalent to 100 meters
pipe_lengths = np.array([25.05 * (100 / 3.15), # path 1
                         27.8 * (100 / 3.15),  # path 2
                         26 * (100 / 3.15),    # path 3
                         23.25 * (100 / 3.15)])# path 4
                                                # pipe lengths, m
    
bends_per_path = np.array([4 + 3, 8 + 3, 4 + 4, 5 + 4])  # number of bends per path

K = np.array([4*0.4 + 3*0.9,            # path 1 -> 4 45 and 3 90 degrees
                         8*0.4 + 3*0.9, # path 2 -> 8 45 and 3 90 degrees
                         4*0.4 + 4*0.9, # path 3 -> 4 45 and 4 90 degrees
                         5*0.4 + 4*0.9])# path 4 -> 5 45 and 4 90 degrees
                                                # minor loss coefficient for each path
    
pipe_diameters = np.array([2 / 39.37,  # 2" to m HDPE (pipe 1)
                           1.5 / 39.37,  # 1.5" to m HDPE (pipe 2)
                           2 / 39.37,  # 2" to m PVC (pipe 3)
                           1.5 / 39.37])  # 1.5" to m PVC (pipe 4)
                                            # pipe diameter, m
    
pipe_roughness = np.array([8.116e-6,  # HDPE (pipe 1)
                           8.116e-6,  # HDPE (pipe 2)
                           3.334e-6,  # PVC (pipe 3)
                           3.334e-6]) # PVC (pipe 4)
                                        # pipe roughness, m

    
water_density = 1e3  # density of water, kg/m^3
water_viscosity = 1e-3  # dynamic viscosity, Pa*s

# Cross-sectional area of the pipe (in m^2)
pipe_areas = np.pi * (pipe_diameters / 2) ** 2
pipe_areas_entrance = np.pi * (0.1 / 2) ** 2

 Now we will define a function that will brute force a value for the friction factor:
 

\begin{equation}
\frac{1}{\sqrt{f}} = -2.0 \times \log_{10} \left( \frac{(\epsilon / d)}{3.7} + \frac{2.51}{\text{Re} \cdot \sqrt{f}} \right)
\end{equation}


In [2]:
def implicit_f(f, relative_roughness, Re):
    return 1 / np.sqrt(f) + 2.0 * np.log10((relative_roughness / 3.7) + (2.51 / (Re * np.sqrt(f))))

We will define a function to calculate head loss:

\begin{align}
h_{\text{losses}} &= \frac{Q^2}{2gA^2} \left( f \frac{L}{d} + K \right)
\end{align}

In [3]:
def pipe_head_loss(f, length, k, Q, pipe_area, pipe_diameter):
    return (((f * length / pipe_diameter) + k) * (Q**2 / (2 * g * pipe_area**2)))

Now we will calculate pump head given:

\begin{align}
h_p &= \frac{\left( \frac{Q^2}{(2g{A^2})} + z_{\text{out}} - z_{\text{in}} + h_{\text{losses}} \right)}{\eta}
\end{align}

In [4]:
def pump_head(Q, pipe_area, z_in, z_out, h_losses, g, efficiency):
    velocity_head = (Q / pipe_area) ** 2 / (2 * g)
    elevation_head = z_out - z_in
    return (velocity_head + elevation_head + h_losses) / efficiency

Now we will calculate power provided by pump/motor:

\begin{align}
\dot{W} &= h_p g \dot{m} \\
  &= h_p g \rho Q
\end{align}

In [5]:
def power_by_pump(h_p, g, rho, Q):
    return h_p * g * rho * Q  # Power in Watts

Total cost is cost per energy (0.15USD/kWh), plus cost for each fitting/bend is 18USD, and cost of pipes (pipe 1, 2, 3, and 4, are 15, 11, 18, 15 USD/m respectively):

\begin{align}
\text{cost} &= \dot{W} \times \text{hours in a year} \times \text{years} \times \text{cost per unit energy} \\
            &\quad + L \times \text{cost per meter of pipe} + \text{number of bends} \times \text{cost per bend}
\end{align}

In [6]:
cost_per_energy = 0.15  # USD/kWh
cost_per_bend = 18  # USD per bend
pipe_costs = np.array([15, 11, 18, 15])  # Cost per meter of pipe for each pipe [USD/m]
years = 10  # Duration in years

def calculate_cost(pipe_length, pipe_cost, bends, bend_cost, energy_cost):
    return pipe_length * pipe_cost + bends * bend_cost + energy_cost

Now we will iterate over velocities to determine flow rates and corresponding costs by finding Q value that satisfies the following:

\begin{align}
h_p &= \frac{\left( \frac{Q^2}{(2g{A^2})} + z_{\text{out}} - z_{\text{in}} + h_{\text{losses}} \right)}{\eta}
=(-(4.06e-4) Q^2 + 8.26e-2 Q + 119.0) [m]\end{align}

Where Q is in m^3/hr

In [7]:
velocities = np.linspace(1, 5, 1000)  # velocity range in m/s
results = []

for path_idx, (length, K_path, bends) in enumerate(zip(pipe_lengths, K, bends_per_path)):
    path_results = []
    for pipe_idx, (area, diameter, cost_per_meter, roughness) in enumerate(zip(pipe_areas, pipe_diameters, pipe_costs, pipe_roughness)):
        min_diff = float('inf')  # initialize minimum difference to a large number
        optimal_h_p = None
        optimal_Q = None
        optimal_cost = None
        
        for velocity in velocities:
            differences = []
            
            # Calculate Q for the specific pipe
            Q_t = velocity * area  
            
            # Calculate Reynolds number for the specific pipe
            Re_t = (water_density * velocity * diameter) / water_viscosity
            
            # Calculate relative roughness for the specific pipe
            relative_roughness = roughness / diameter
            
            # Solve for friction factor for the specific pipe
            f_t = fsolve(implicit_f, 0.02, args=(relative_roughness, Re_t))[0]
            
            # Solve for entrance friction factor
            f_entrance_t = fsolve(implicit_f, 0.02, args=(3.334e-6 / 0.1, (water_density * velocity * 0.1) / water_viscosity))[0]
            
            # Calculate head loss for the specific pipe
            head_loss_t = pipe_head_loss(f_t, length, K_path, Q_t, area, diameter)
            
            # Calculate entrance head loss
            entrance_loss_t = pipe_head_loss(f_entrance_t, 30, 1.765, Q_t, pipe_areas_entrance, 0.1)
            
            # Total head loss
            total_loss_t = head_loss_t + entrance_loss_t
            
            # Calculate pump head for this pipe
            h_p = pump_head(Q_t, area, z_in, z_out, total_loss_t, g, efficiency)
            
            # Check pump head using given formula
            h_p_check = -4.06e-4 * (Q_t/3600)**2 + 8.26e-2 * (Q_t/3600) + 119
            
            # Calculate the absolute difference
            diff = abs(h_p - h_p_check)
            differences.append(diff)
            
            # Track minimum difference and corresponding values to find Q where h_p_check≈h_p
            if diff < min_diff:
                min_diff = diff
                optimal_h_p = h_p
                optimal_Q = Q_t
        
        # After iterating through all velocities, we now know the optimal values
        power_needed = power_by_pump(optimal_h_p, g, water_density, optimal_Q)  # in Watts
            
        # Calculate energy cost over 10 years
        hours_of_use_per_day = (23*3000/(8.64e7))*24/optimal_Q # Hrs to reach avg. of 23 L/day per plant (3000 plants)
        energy_cost = power_needed / 1000 * hours_of_use_per_day * 365 * years * cost_per_energy  # in USD
        
        # Calculate total cost (including pipe, bends, and energy cost)
        total_cost = calculate_cost(length, cost_per_meter, bends, cost_per_bend, energy_cost)
            
        # Append results for this pipe with the optimal combination
        path_results.append({
            'path': path_idx + 1,
            'pipe': pipe_idx + 1,
            'Q': optimal_Q,
            'Re': Re_t,
            'relative_roughness': relative_roughness,
            'head_loss': total_loss_t,
            'pump_head': optimal_h_p,
            'power': power_needed,
            'energy_cost': energy_cost,
            'total_cost': total_cost
        })
        
    # Append results for the path
    results.append(path_results)
    
# Loop through the results to display all optimal details for each path and pipe
for path in results:
    print(f"Path {path[0]['path']} details:")
    for pipe in path:
        # Display all information for the current pipe
        print(f"  Pipe: {pipe['pipe']}")
        print(f"    Flow Rate (Q): {pipe['Q']:.10f} m³/s")
        print(f"    Reynolds Number (Re): {pipe['Re']:.4f}")
        print(f"    Relative Roughness: {pipe['relative_roughness']:.6f}")
        print(f"    Head Loss: {pipe['head_loss']:.4f} m")
        print(f"    Pump Head: {pipe['pump_head']:.4f} m")
        print(f"    Power Required: {pipe['power']:.4f} Watts")
        print(f"    Energy Cost (10 years): ${pipe['energy_cost']:.2f}")
        print(f"    Total Cost (10 years): ${pipe['total_cost']:.2f}")
    print("-" * 50)

    
# Extract most optimal result (according to minimal cost over 10 years of operation)
optimal_result = min([pipe for path in results for pipe in path], key=lambda x: x['total_cost'])

# Display most optimal path and pipe details (according to minimal cost over 10 years of operation)
print("-" * 50)
print("Optimal Path and Pipe:")
print(f"Path: {optimal_result['path']}")
print(f"Pipe: {optimal_result['pipe']}")
print(f"Flow Rate (Q): {optimal_result['Q']:.10f} m³/s")
print(f"Total Cost (10 years): ${optimal_result['total_cost']:.2f}")
print(f"Pump Time (10 years): {((optimal_result['energy_cost']*1000/optimal_result['power'])/0.15/24):.0f} days")

Path 1 details:
  Pipe: 1
    Flow Rate (Q): 0.0045994412 m³/s
    Reynolds Number (Re): 254000.5080
    Relative Roughness: 0.000160
    Head Loss: 330.7337 m
    Pump Head: 119.0773 m
    Power Required: 5372.8305 Watts
    Energy Cost (10 years): $12258.26
    Total Cost (10 years): $24312.83
  Pipe: 2
    Flow Rate (Q): 0.0021398205 m³/s
    Reynolds Number (Re): 190500.3810
    Relative Roughness: 0.000213
    Head Loss: 465.6049 m
    Pump Head: 118.8629 m
    Power Required: 2495.1274 Watts
    Energy Cost (10 years): $12236.19
    Total Cost (10 years): $21109.81
  Pipe: 3
    Flow Rate (Q): 0.0046643650 m³/s
    Reynolds Number (Re): 254000.5080
    Relative Roughness: 0.000066
    Head Loss: 315.6279 m
    Pump Head: 118.8738 m
    Power Required: 5439.3588 Watts
    Energy Cost (10 years): $12237.31
    Total Cost (10 years): $26677.59
  Pipe: 4
    Flow Rate (Q): 0.0021717752 m³/s
    Reynolds Number (Re): 190500.3810
    Relative Roughness: 0.000088
    Head Loss: 443.1536

According to the results above, we notice that path 4 with pipe 2 (1.5" HDPE) provides the cheapest option for a 10 year operation/maintnance of 20,510.04 USD.